# SF3D API Server - Fast Installation (v4)

**OPTIMIZED FOR SPEED**: Uses `/tmp` (local storage) instead of home directory (network storage)

## Setup Steps:
1. Run Cell 1: Set up fast local environment
2. Run Cell 2: Install dependencies (much faster!)
3. **RESTART KERNEL**
4. Run Cell 3: Load SF3D model
5. Run Cell 4: Define API
6. Run Cell 5: Start server

**Why faster?**
- Your home directory (`~`) is network-mounted (NFS) → slow writes
- `/tmp` is local SSD on the GPU machine → fast writes
- All packages and models go to `/tmp`

## Cell 1: Setup Fast Local Environment

**Creates virtual environment in /tmp (fast local storage)**

In [ ]:
import os
import sys
from pathlib import Path

# Use /tmp for fast local storage
username = os.environ.get('USER', 'user')
WORK_DIR = Path(f"/tmp/sf3d_{username}")
VENV_DIR = WORK_DIR / "venv"
SF3D_DIR = WORK_DIR / "stable-fast-3d"
CACHE_DIR = WORK_DIR / "cache"

# Create directories
WORK_DIR.mkdir(exist_ok=True)
CACHE_DIR.mkdir(exist_ok=True)

print(f"Working directory: {WORK_DIR}")
print(f"Virtual env: {VENV_DIR}")
print(f"SF3D repo: {SF3D_DIR}")
print(f"Cache: {CACHE_DIR}")
print("\n✅ Directories created in /tmp (fast local storage)")

## Cell 2: Install Everything to /tmp

**This should be MUCH faster than installing to home directory**

In [ ]:
%%bash -s "$VENV_DIR" "$CACHE_DIR" "$SF3D_DIR"
VENV_DIR=$1
CACHE_DIR=$2
SF3D_DIR=$3

echo "Installing to: $VENV_DIR"
echo "Using cache: $CACHE_DIR"
echo ""

# Create virtual environment if it doesn't exist
if [ ! -d "$VENV_DIR" ]; then
    echo "Creating virtual environment..."
    python -m venv "$VENV_DIR"
    echo "✅ Virtual environment created"
else
    echo "✅ Virtual environment already exists"
fi

# Activate venv
source "$VENV_DIR/bin/activate"

# Set pip cache to /tmp
export PIP_CACHE_DIR="$CACHE_DIR/pip"
export HF_HOME="$CACHE_DIR/huggingface"

echo ""
echo "Installing packages to /tmp (this is FAST!)..."
echo ""

# Upgrade pip first
pip install --upgrade pip setuptools wheel

# Install PyTorch with CUDA 11.8
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Install FastAPI
pip install fastapi uvicorn[standard] python-multipart

# Install core dependencies
pip install pillow numpy omegaconf einops

# Install 3D libraries
pip install trimesh pymeshlab

# Install ML libraries
pip install transformers accelerate safetensors huggingface-hub

# Install additional deps
pip install opencv-python imageio rembg

echo ""
echo "✅ All packages installed!"
echo ""

# Clone SF3D if not already cloned
if [ ! -d "$SF3D_DIR" ]; then
    echo "Cloning SF3D repository..."
    git clone https://github.com/Stability-AI/stable-fast-3d.git "$SF3D_DIR"
    echo "✅ SF3D cloned"
else
    echo "✅ SF3D already cloned"
fi

echo ""
echo "✅ Installation complete!"
echo ""
echo "⚠️  IMPORTANT: Now go to Kernel → Restart Kernel"
echo "   Then run Cell 3 to load the model."

## Cell 3: Load SF3D Model

**After restarting kernel, run this cell**

In [ ]:
import os
import sys
import time
from pathlib import Path

# Reconstruct paths
username = os.environ.get('USER', 'user')
WORK_DIR = Path(f"/tmp/sf3d_{username}")
VENV_DIR = WORK_DIR / "venv"
SF3D_DIR = WORK_DIR / "stable-fast-3d"
CACHE_DIR = WORK_DIR / "cache"

# Add venv to path (so imports work)
venv_site_packages = VENV_DIR / "lib" / "python3.10" / "site-packages"
if not venv_site_packages.exists():
    # Try python3.11
    venv_site_packages = VENV_DIR / "lib" / "python3.11" / "site-packages"

sys.path.insert(0, str(venv_site_packages))
sys.path.insert(0, str(SF3D_DIR))

print(f"Using venv: {venv_site_packages}")
print(f"Using SF3D: {SF3D_DIR}")

# Set Hugging Face cache to /tmp
os.environ['HF_HOME'] = str(CACHE_DIR / 'huggingface')
os.environ['TRANSFORMERS_CACHE'] = str(CACHE_DIR / 'huggingface' / 'transformers')
os.environ['HF_DATASETS_CACHE'] = str(CACHE_DIR / 'huggingface' / 'datasets')

print(f"HF cache: {os.environ['HF_HOME']}")

# Import libraries
import torch
import numpy as np
from PIL import Image

print("\n✅ Libraries imported")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

# Import SF3D
print("\nImporting SF3D...")
try:
    from sf3d.system import SF3D
    print("✅ SF3D imported")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("\nTrying alternative...")
    sys.path.insert(0, str(SF3D_DIR / "sf3d"))
    from system import SF3D
    print("✅ SF3D imported (alternative)")

# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\nDevice: {device}")
print("Loading SF3D model (downloads to /tmp/cache)...")
print("First time: ~2GB download, may take 30-60 seconds")

start_time = time.time()
model = SF3D.from_pretrained(
    "stabilityai/stable-fast-3d",
    config_name="config.yaml",
    weight_name="model.safetensors",
)
model = model.to(device)
model.eval()

print(f"\n✅ SF3D model loaded in {time.time() - start_time:.1f}s")
print(f"   Model: stabilityai/stable-fast-3d")
print(f"   Device: {device}")

# Output directory
output_dir = WORK_DIR / "outputs"
output_dir.mkdir(exist_ok=True)
print(f"\nOutput dir: {output_dir}")
print("\n✅ Ready to generate meshes!")

## Cell 4: Define API

In [ ]:
import io
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

app = FastAPI(
    title="Stable Fast 3D API",
    description="Generate 3D meshes using SF3D (optimized for speed)",
    version="4.0.0"
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def root():
    return {
        "message": "SF3D API (Fast Installation)",
        "status": "running",
        "device": device,
        "storage": "local /tmp (fast)",
        "model": "stabilityai/stable-fast-3d"
    }

@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "device": device,
        "cuda_available": torch.cuda.is_available(),
        "model_loaded": model is not None
    }

@app.post("/generate")
async def generate_mesh(
    file: UploadFile = File(...),
    texture_resolution: int = Form(1024),
    remesh_option: str = Form("none"),
    foreground_ratio: float = Form(0.85)
):
    try:
        image_data = await file.read()
        image = Image.open(io.BytesIO(image_data))
        
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        print(f"\n[{time.strftime('%H:%M:%S')}] Processing: {image.size}")
        
        start_time = time.time()
        
        with torch.no_grad():
            output = model.run(
                image,
                bake_resolution=texture_resolution,
                remesh=remesh_option if remesh_option != 'none' else None,
                vertex_count=-1,
            )
        
        gen_time = time.time() - start_time
        print(f"[{time.strftime('%H:%M:%S')}] Generated in {gen_time:.2f}s")
        
        timestamp = int(time.time() * 1000)
        output_path = output_dir / f"mesh_{timestamp}.glb"
        
        mesh = output['mesh'] if isinstance(output, dict) else output
        mesh.export(str(output_path))
        
        file_size = output_path.stat().st_size
        print(f"[{time.strftime('%H:%M:%S')}] Saved: {file_size / 1024:.1f} KB\n")
        
        return FileResponse(
            path=output_path,
            media_type="model/gltf-binary",
            filename=f"mesh_{timestamp}.glb",
            headers={
                "X-Generation-Time": str(gen_time),
                "X-File-Size": str(file_size)
            }
        )
    
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=str(e))

print("✅ API endpoints defined")

## Cell 5: Start Server

In [ ]:
PORT = 8765
HOST = "0.0.0.0"

print("="*70)
print("🚀 SF3D API Server (Fast /tmp Installation)")
print("="*70)
print(f"URL: http://itp-ml.itp.tsoa.nyu.edu:{PORT}/")
print(f"Device: {device}")
print(f"Storage: /tmp (local SSD - FAST)")
print(f"Model: SF3D")
print("="*70)
print(f"\nTest: python tests/sf3d_api_client.py <image> --server http://itp-ml.itp.tsoa.nyu.edu:{PORT}")
print("\n⚠️  KEEP THIS CELL RUNNING\n")

try:
    uvicorn.run(app, host=HOST, port=PORT, log_level="info")
except KeyboardInterrupt:
    print("\n✅ Server stopped")